In [1]:
import pandas as pd
import numpy as np
import os
import acquire_nick as get
import prepare_nick as prep
import re
import wrangle as w

In [11]:
df = get.acquire_data_new()

Using cached CSV


In [2]:
df = w.wrangle_df()

Using cached CSV


In [3]:
df

,budget,genres,id,imdb_id,overview,popularity,production_companies,production_countries,revenue,runtime,title,vote_average,vote_count,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,release_year,profitable
release_date,,,,,,,,,,,,,,,,,,,
1915-02-08,10000000.0,Drama History War,618,tt0004972,"The Birth of A Nation is a silent film from 1915 and the highest grossing silent film in film history. The film tells a romance story during the American civil war. D.W. Griffith invested heavily in its high production values, pioneering many new camera effects. The Birth of a Nation was strongly protested for its negative portrayal of newly freed slaves (mostly white actors in blackface), which went on to create and propagate negative images of blacks of the early 20th century in the United States. The film was used as a recruiting propaganda by the KKK until the 1940s.",5.113205,EpochFilmCo.,UnitedStatesofAmerica,11000000.0,193.0,The Birth of a Nation,6.4,109.0,Lillian Gish,Mae Marsh,Henry B. Walthall,49.0,1915,False
1915-12-13,10000000.0,Drama,70368,tt0005078,"A venal, spoiled stockbroker's wife impulsively embezzles $10,000 from the charity she chairs and desperately turns to a Burmese ivory trader to replace the stolen money.",0.680407,JesseL.LaskyFeaturePlayCompany,UnitedStatesofAmerica,137365.0,59.0,The Cheat,4.8,11.0,Fannie Ward,Jack Dean,Sessue Hayakawa,11.0,1915,False
1916-12-24,10000000.0,Adventure Drama Action Science Fiction,30266,tt0006333,"Captain Nemo has built a fantastic submarine for his mission of revenge. He has traveled over 20,000 leagues in search of Charles Denver - a man who caused the death of Princess Daaker. Seeing what he had done, Denver took the daughter to his yacht and sailed away.",0.917777,TheUniversalFilmMfg.Co.,Bahamas UnitedStatesofAmerica,8000000.0,105.0,"20,000 Leagues Under the Sea",6.0,7.0,Lois Alexander,Curtis Benton,Wallace Clarke,13.0,1916,False
1918-08-01,10000000.0,Comedy Drama,54242,tt0009369,"Mickey, an orphan who has been brought up in a mining settlement, is sent to New York to live with her aunt.",0.211008,MabelNormandFeatureFilmCompany,UnitedStatesofAmerica,8000000.0,93.0,Mickey,7.5,4.0,Mabel Normand,George Nichols,Wheeler Oakman,8.0,1918,False
1921-01-21,10000000.0,Comedy Drama,10098,tt0012349,"Considered one of Charlie Chaplin's best films, The Kid also made a star of little Jackie Coogan, who plays a boy cared for by The Tramp when he's abandoned by his mother, Edna. Later, Edna has a change of heart and aches to be reunited with her son. When she finds him and wrests him from The Tramp, it makes for what turns out be one of the most heart-wrenching scenes ever included in a comedy.",8.168456,CharlesChaplinProductions FirstNationalPictures,UnitedStatesofAmerica,2500000.0,68.0,The Kid,8.0,404.0,Charlie Chaplin,Jackie Coogan,Edna Purviance,35.0,1921,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-03,11000000.0,Action Crime Mystery Thriller,395834,tt5362988,An FBI agent teams with the town's veteran game tracker to investigate a murder that occurred on a Native American reservation.,40.796775,ThunderRoadPictures StarThrowerEntertainment AcaciaFilmedEntertainment,UnitedKingdom UnitedStatesofAmerica Canada,184770205.0,111.0,Wind River,7.4,181.0,Jeremy Renner,Elizabeth Olsen,Gil Birmingham,22.0,2017,True
2017-08-03,60000000.0,Action Western Science Fiction Fantasy Horror,353491,tt1648190,"The last Gunslinger, Roland Deschain, has been locked in an eternal battle with Walter O’Dim, also known as the Man in Black, determined to prevent him from toppling the Dark Tower, which holds the universe together. With the fate of the worlds at stake, good and evil will collide in the ultimate battle as only Roland can defend the Tower from the Man in Black.",50.903593,ImagineEntertainment WeedRoadPictures MediaRightsCapital SonyPicturesEntertainment,SouthAfrica UnitedStatesofAmerica,71000000.0,95.0,The Dark Tower,5.7,688.0,Idris Elba,Matthew McConaughe

In [13]:
def prep_data(df):
    
    # drop columns that do not inform our decision-making process
    df.drop(columns=['Unnamed: 0', 'adult', 'belongs_to_collection',
                 'homepage', 'original_language', 'original_title',
                 'poster_path', 'spoken_languages', 'status', 'tagline', 'video', 'cast',
                 'crew'], inplace=True)
    # drop all na values (sought other method, but it wasn't more effective
    df.dropna(axis=0, inplace=True)
    # by first sorting df descending from budget, it will keep more valuable values in next step
    df = df.sort_values(by='budget', ascending=False, na_position='last')
    # keep only the first instances of duplicates, now that they are sorted. 
    df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)
    df['genres'] = df['genres'].apply(lambda x: ' '.join([i['name'] for i in eval(x)]))
    # fills values less than one million with the median value, 10000000
    df['budget'] = np.where(df['budget'].between(0,1000000), df['budget'].median(), df['budget'])
    # extract the release year before converting to date-time
    df['release_year'] = df.release_date.apply(lambda x : x.split('-')[0]).astype(int)
    # convert to a datetime format. Consider to_datetime as well. 
    df['release_date'] = df['release_date'].astype('datetime64[ns]')
    # movie is successful if (revenue >= budget * 2)
    for index, row in df.iterrows():
        try:
            budget = df.at[index, 'budget']
            revenue = df.at[index, 'revenue']
            if (revenue >= budget * 2):
                profitable = True
            else:
                profitable = False
        except:     # if budget or revenue is empty
            profitable = np.nan
    
        df.at[index, 'profitable'] = profitable
    # convert from object to bool. May set to binary later.  
    df['profitable'] = df['profitable'].astype(bool)
    
    # Set 'release_date' as index & sort values
    df = df.set_index('release_date').sort_index()
    
    return df

In [14]:
df = prep_data(df)

In [15]:
df

,budget,genres,id,imdb_id,overview,popularity,production_companies,production_countries,revenue,runtime,title,vote_average,vote_count,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,release_year,profitable
release_date,,,,,,,,,,,,,,,,,,,
1915-02-08,10000000.0,Drama History War,618,tt0004972,"The Birth of A Nation is a silent film from 1915 and the highest grossing silent film in film history. The film tells a romance story during the American civil war. D.W. Griffith invested heavily in its high production values, pioneering many new camera effects. The Birth of a Nation was strongly protested for its negative portrayal of newly freed slaves (mostly white actors in blackface), which went on to create and propagate negative images of blacks of the early 20th century in the United States. The film was used as a recruiting propaganda by the KKK until the 1940s.",5.113205,EpochFilmCo.,UnitedStatesofAmerica,11000000.0,193.0,The Birth of a Nation,6.4,109.0,Lillian Gish,Mae Marsh,Henry B. Walthall,49.0,1915,False
1915-12-13,10000000.0,Drama,70368,tt0005078,"A venal, spoiled stockbroker's wife impulsively embezzles $10,000 from the charity she chairs and desperately turns to a Burmese ivory trader to replace the stolen money.",0.680407,JesseL.LaskyFeaturePlayCompany,UnitedStatesofAmerica,137365.0,59.0,The Cheat,4.8,11.0,Fannie Ward,Jack Dean,Sessue Hayakawa,11.0,1915,False
1916-12-24,10000000.0,Adventure Drama Action Science Fiction,30266,tt0006333,"Captain Nemo has built a fantastic submarine for his mission of revenge. He has traveled over 20,000 leagues in search of Charles Denver - a man who caused the death of Princess Daaker. Seeing what he had done, Denver took the daughter to his yacht and sailed away.",0.917777,TheUniversalFilmMfg.Co.,Bahamas UnitedStatesofAmerica,8000000.0,105.0,"20,000 Leagues Under the Sea",6.0,7.0,Lois Alexander,Curtis Benton,Wallace Clarke,13.0,1916,False
1918-08-01,10000000.0,Comedy Drama,54242,tt0009369,"Mickey, an orphan who has been brought up in a mining settlement, is sent to New York to live with her aunt.",0.211008,MabelNormandFeatureFilmCompany,UnitedStatesofAmerica,8000000.0,93.0,Mickey,7.5,4.0,Mabel Normand,George Nichols,Wheeler Oakman,8.0,1918,False
1921-01-21,10000000.0,Comedy Drama,10098,tt0012349,"Considered one of Charlie Chaplin's best films, The Kid also made a star of little Jackie Coogan, who plays a boy cared for by The Tramp when he's abandoned by his mother, Edna. Later, Edna has a change of heart and aches to be reunited with her son. When she finds him and wrests him from The Tramp, it makes for what turns out be one of the most heart-wrenching scenes ever included in a comedy.",8.168456,CharlesChaplinProductions FirstNationalPictures,UnitedStatesofAmerica,2500000.0,68.0,The Kid,8.0,404.0,Charlie Chaplin,Jackie Coogan,Edna Purviance,35.0,1921,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-03,11000000.0,Action Crime Mystery Thriller,395834,tt5362988,An FBI agent teams with the town's veteran game tracker to investigate a murder that occurred on a Native American reservation.,40.796775,ThunderRoadPictures StarThrowerEntertainment AcaciaFilmedEntertainment,UnitedKingdom UnitedStatesofAmerica Canada,184770205.0,111.0,Wind River,7.4,181.0,Jeremy Renner,Elizabeth Olsen,Gil Birmingham,22.0,2017,True
2017-08-03,60000000.0,Action Western Science Fiction Fantasy Horror,353491,tt1648190,"The last Gunslinger, Roland Deschain, has been locked in an eternal battle with Walter O’Dim, also known as the Man in Black, determined to prevent him from toppling the Dark Tower, which holds the universe together. With the fate of the worlds at stake, good and evil will collide in the ultimate battle as only Roland can defend the Tower from the Man in Black.",50.903593,ImagineEntertainment WeedRoadPictures MediaRightsCapital SonyPicturesEntertainment,SouthAfrica UnitedStatesofAmerica,71000000.0,95.0,The Dark Tower,5.7,688.0,Idris Elba,Matthew McConaughe